### Task at hand:
1) read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html]
2) read additional information abount NFL teams (football, in [assets/nfl.csv]
3) read additional information abount MLB (baseball, in [assets/mlb.csv]
4) read additional information abount NBA (basketball, in [assets/nba.csv]
5) read additional information abount NHL (hockey, in [assets/nhl.csv]

For each type of sport we find win/loss ratio's correlation with the population of the city it is in
We will use data strictly from year 2018 for this analysis
We will focus only on BIG 4 (no data related to BIG will be included)


In [2]:
import pandas as pd
import numpy as np

In [3]:
# Preparing wiki data
cities = pd.read_html("./assets/wikipedia_data.html")
cities = cities[1].iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={"Population (2016 est.)[8]" : "Population (2016 est.)"}, inplace=True)
cities["Population (2016 est.)"] = cities["Population (2016 est.)"].map(lambda x: float(x))
cities.replace(r"\[note [\d][\d]?\]", "", regex=True, inplace=True)

cities.head()

,Metropolitan area,Population (2016 est.),NFL,MLB,NBA,NHL
0,New York City,20153634.0,Giants Jets,Yankees Mets,Knicks Nets,Rangers Islanders Devils
1,Los Angeles,13310447.0,Rams Chargers,Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982.0,49ers Raiders,Giants Athletics,Warriors,Sharks
3,Chicago,9512999.0,Bears,Cubs White Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323.0,Cowboys,Rangers,Mavericks,Stars


In [355]:
# Additional NFL data
nfl_df = pd.read_csv("./assets/nfl.csv")
nfl_df = nfl_df[nfl_df["year"] == 2018]
nfl_df.drop(index=[0, 5, 10, 15, 20, 25, 30, 35], inplace=True)

# convert str to numbers for dedicated columns
colums_to_float = ['DSRS', 'L', 'MoV', 'OSRS', 'PA', 'PD', 'PF', 'SRS', 'SoS', 'T', 'W', 'W-L%']
nfl_df[colums_to_float] = nfl_df[colums_to_float].apply(lambda x: x.map(lambda y : float(y)))
nfl_df["team"].replace("\*|\+", "", regex=True, inplace=True)

pattern_2 = r"(?P<Metropolitan_area>.+ )(?P<team_label>[A-Z\d][a-z\d]+)"
nfl_df["label"] = nfl_df["team"].str.extract(pattern_2)["team_label"]
cities.replace(r"\[note [\d][\d]?\]", "", regex=True, inplace=True)

nfl_areas = []
for i in range(len(nfl_df["label"])):
    for j in range(len(cities["NFL"])):
        if nfl_df.iloc[i]["label"] in cities["NFL"][j]:
            nfl_areas.append(cities["Metropolitan area"][j])
nfl_df["Metropolitan area"] = nfl_areas

nfl_df["Win_Loss_Ratio"] = nfl_df["W"] / (nfl_df["W"] + nfl_df["L"])


In [356]:
nfl_df_question = nfl_df.groupby("Metropolitan area")["Win_Loss_Ratio"].mean()
result = pd.merge(nfl_df_question.reset_index(), cities[["Metropolitan area", "Population (2016 est.)"]],
                   how="left", on="Metropolitan area")


# pass in metropolitan area population from cities
population_by_region = list(result["Population (2016 est.)"]) 

# pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
win_loss_by_region = list(result["Win_Loss_Ratio"])

len(population_by_region)

29

In [484]:
mlb_df = pd.read_csv("./assets/mlb.csv")
mlb_df = mlb_df[mlb_df["year"] == 2018]

pattern = r"(?P<Metropolitan_area>.+ )(?P<team_label>[A-Z\d][a-z\d]+)"
mlb_df["label"] = mlb_df["team"].str.extract(pattern)["team_label"]
# Label correction:
mlb_df.at[0, "label"] = "Red Sox"
mlb_df.at[8, "label"] = "White Sox"

# Setting Metropolitan area
mlb_areas = []
for i in range(len(mlb_df["label"])):
    for j in range(len(cities["MLB"])):
        if mlb_df.iloc[i]["label"] in cities["MLB"][j]:
            mlb_areas.append(cities["Metropolitan area"][j])
mlb_df["Metropolitan area"] = mlb_areas

# Win/Loss ratio
mlb_df["Win_Loss_Ratio"] = mlb_df["W"] / (mlb_df["W"] + mlb_df["L"])
mlb_df[["team", "Metropolitan area", "Win_Loss_Ratio"]]


,team,Metropolitan area,Win_Loss_Ratio
0,Boston Red Sox,Boston,0.666667
1,New York Yankees,New York City,0.617284
2,Tampa Bay Rays,Tampa Bay Area,0.555556
3,Toronto Blue Jays,Toronto,0.450617
4,Baltimore Orioles,Baltimore,0.290123
5,Cleveland Indians,Cleveland,0.561728
6,Minnesota Twins,Minneapolis–Saint Paul,0.481481
7,Detroit Tigers,Detroit,0.395062
8,Chicago White Sox,Chicago,0.382716
9,Kansas City Royals,Kansas City,0.358025


In [481]:
mlb_df_question = mlb_df.groupby("Metropolitan area")["Win_Loss_Ratio"].mean()
result = pd.merge(mlb_df_question.reset_index(), cities[["Metropolitan area", "Population (2016 est.)"]],
                   how="left", on="Metropolitan area")


population_by_region = list(result["Population (2016 est.)"]) # pass in metropolitan area population from cities
win_loss_by_region = list(result["Win_Loss_Ratio"]) # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

result

,Metropolitan area,Win_Loss_Ratio,Population (2016 est.)
0,Atlanta,0.555556,5789700.0
1,Baltimore,0.290123,2798886.0
2,Boston,0.666667,4794447.0
3,Chicago,0.482769,9512999.0
4,Cincinnati,0.413580,2165139.0
5,Cleveland,0.561728,2055612.0
6,Dallas–Fort Worth,0.413580,7233323.0
7,Denver,0.558282,2853077.0
8,Detroit,0.395062,4297617.0
9,Houston,0.635802,6772470.0


In [415]:
nba_df = pd.read_csv("./assets/nba.csv")
nba_df = nba_df[nba_df["year"] == 2018]
nba_df["team"].replace("\*?\s\([\d][\d]?\)", "", regex=True, inplace=True)

pattern = r"(?P<Metropolitan_area>.+ )(?P<team_label>[A-Z\d][a-z\d]+)"
nba_df["label"] = nba_df["team"].str.extract(pattern)["team_label"]

# Setting Metropolitan area
nba_areas = []
for i in range(len(nba_df["label"])):
    for j in range(len(cities["NBA"])):
        if nba_df.iloc[i]["label"] in cities["NBA"][j]:
            nba_areas.append(cities["Metropolitan area"][j])
nba_df["Metropolitan area"] = nba_areas

# Win/Loss ratio
# convert str to numbers for dedicated columns
colums_to_float = ["L", "W"]
nba_df[colums_to_float] = nba_df[colums_to_float].apply(lambda x: x.map(lambda y : float(y)))
nba_df["Win_Loss_Ratio"] = nba_df["W"] / (nba_df["W"] + nba_df["L"])


In [418]:
nba_df_question = nba_df.groupby("Metropolitan area")["Win_Loss_Ratio"].mean()
result = pd.merge(nba_df_question.reset_index(), cities[["Metropolitan area", "Population (2016 est.)"]],
                   how="left", on="Metropolitan area")


population_by_region = list(result["Population (2016 est.)"]) # pass in metropolitan area population from cities
win_loss_by_region = list(result["Win_Loss_Ratio"]) # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

len(population_by_region)

28

In [428]:
nhl_df = pd.read_csv("./assets/nhl.csv")
nhl_df = nhl_df[nhl_df["year"] == 2018]
nhl_df.drop(index=[0, 9, 18, 26], inplace=True)
nhl_df["team"].replace("\*", "", regex=True, inplace=True)

pattern = r"(?P<Metropolitan_area>.+ )(?P<team_label>[A-Z\d][a-z\d]+)"
nhl_df["label"] = nhl_df["team"].str.extract(pattern)["team_label"]

# Setting Metropolitan area
nhl_areas = []
for i in range(len(nhl_df["label"])):
    for j in range(len(cities["NHL"])):
        if nhl_df.iloc[i]["label"] in cities["NHL"][j]:
            nhl_areas.append(cities["Metropolitan area"][j])
nhl_df["Metropolitan area"] = nhl_areas

# Win/Loss ratio
# convert str to numbers for dedicated columns
colums_to_float = ["L", "W"]
nhl_df[colums_to_float] = nhl_df[colums_to_float].apply(lambda x: x.map(lambda y : float(y)))
nhl_df["Win_Loss_Ratio"] = nhl_df["W"] / (nhl_df["W"] + nhl_df["L"])


In [429]:
nhl_df_question = nhl_df.groupby("Metropolitan area")["Win_Loss_Ratio"].mean()
result = pd.merge(nhl_df_question.reset_index(), cities[["Metropolitan area", "Population (2016 est.)"]],
                   how="left", on="Metropolitan area")


population_by_region = list(result["Population (2016 est.)"]) # pass in metropolitan area population from cities
win_loss_by_region = list(result["Win_Loss_Ratio"]) # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

len(population_by_region)

28

### The win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [17]:
import pandas as pd
import scipy.stats as stats

def nhl_correlation(): 
    cities = pd.read_html("./assets/wikipedia_data.html")
    cities = cities[1].iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns={"Population (2016 est.)[8]" : "Population (2016 est.)"}, inplace=True)
    cities["Population (2016 est.)"] = cities["Population (2016 est.)"].map(lambda x: float(x))
    cities.replace(r"\[note [\d][\d]?\]", "", regex=True, inplace=True)

    nhl_df = pd.read_csv("./assets/nhl.csv")
    nhl_df = nhl_df[nhl_df["year"] == 2018]
    nhl_df.drop(index=[0, 9, 18, 26], inplace=True)
    nhl_df["team"].replace("\*", "", regex=True, inplace=True)

    pattern = r"(?P<Metropolitan_area>.+ )(?P<team_label>[A-Z\d][a-z\d]+)"
    nhl_df["label"] = nhl_df["team"].str.extract(pattern)["team_label"]

    # Setting Metropolitan area
    nhl_areas = []
    for i in range(len(nhl_df["label"])):
        for j in range(len(cities["NHL"])):
            if nhl_df.iloc[i]["label"] in cities["NHL"][j]:
                nhl_areas.append(cities["Metropolitan area"][j])
    nhl_df["Metropolitan area"] = nhl_areas

    # Win/Loss ratio
    # convert str to numbers for dedicated columns
    colums_to_float = ["L", "W"]
    nhl_df[colums_to_float] = nhl_df[colums_to_float].apply(lambda x: x.map(lambda y : float(y)))
    nhl_df["Win_Loss_Ratio"] = nhl_df["W"] / (nhl_df["W"] + nhl_df["L"])    

    nhl_df_question = nhl_df.groupby("Metropolitan area")["Win_Loss_Ratio"].mean()
    result = pd.merge(nhl_df_question.reset_index(), cities[["Metropolitan area", "Population (2016 est.)"]],
                       how="left", on="Metropolitan area")

    population_by_region = list(result["Population (2016 est.)"]) # pass in metropolitan area population from cities
    win_loss_by_region = list(result["Win_Loss_Ratio"]) # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
   
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

0.012486162921209923

## The win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [27]:
import pandas as pd
import scipy.stats as stats

def nba_correlation():
    cities = pd.read_html("./assets/wikipedia_data.html")
    cities = cities[1].iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns={"Population (2016 est.)[8]" : "Population (2016 est.)"}, inplace=True)
    cities["Population (2016 est.)"] = cities["Population (2016 est.)"].map(lambda x: float(x))
    cities.replace(r"\[note [\d][\d]?\]", "", regex=True, inplace=True)

    nba_df = pd.read_csv("./assets/nba.csv")
    nba_df = nba_df[nba_df["year"] == 2018]
    nba_df["team"].replace("\*?\s\([\d][\d]?\)", "", regex=True, inplace=True)

    pattern = r"(?P<Metropolitan_area>.+ )(?P<team_label>[A-Z\d][a-z\d]+)"
    nba_df["label"] = nba_df["team"].str.extract(pattern)["team_label"]

    # Setting Metropolitan area
    nba_areas = []
    for i in range(len(nba_df["label"])):
        for j in range(len(cities["NBA"])):
            if nba_df.iloc[i]["label"] in cities["NBA"][j]:
                nba_areas.append(cities["Metropolitan area"][j])
    nba_df["Metropolitan area"] = nba_areas

    # Win/Loss ratio
    # convert str to numbers for dedicated columns
    colums_to_float = ["L", "W"]
    nba_df[colums_to_float] = nba_df[colums_to_float].apply(lambda x: x.map(lambda y : float(y)))
    nba_df["Win_Loss_Ratio"] = nba_df["W"] / (nba_df["W"] + nba_df["L"])

    nba_df_question = nba_df.groupby("Metropolitan area")["Win_Loss_Ratio"].mean()
    result = pd.merge(nba_df_question.reset_index(), cities[["Metropolitan area", "Population (2016 est.)"]],
                       how="left", on="Metropolitan area")

    # pass in metropolitan area population from cities
    population_by_region = list(result["Population (2016 est.)"]) 
    # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    win_loss_by_region = list(result["Win_Loss_Ratio"]) 

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.17657160252844623

## The win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [28]:
import pandas as pd
import scipy.stats as stats

def mlb_correlation(): 
    cities = pd.read_html("./assets/wikipedia_data.html")
    cities = cities[1].iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns={"Population (2016 est.)[8]" : "Population (2016 est.)"}, inplace=True)
    cities["Population (2016 est.)"] = cities["Population (2016 est.)"].map(lambda x: float(x))
    cities.replace(r"\[note [\d][\d]?\]", "", regex=True, inplace=True)

    mlb_df = pd.read_csv("./assets/mlb.csv")
    mlb_df = mlb_df[mlb_df["year"] == 2018]

    pattern = r"(?P<Metropolitan_area>.+ )(?P<team_label>[A-Z\d][a-z\d]+)"
    mlb_df["label"] = mlb_df["team"].str.extract(pattern)["team_label"]
    # Label correction:
    mlb_df.at[0, "label"] = "Red Sox"
    mlb_df.at[8, "label"] = "White Sox"

    # Setting Metropolitan area
    mlb_areas = []
    for i in range(len(mlb_df["label"])):
        for j in range(len(cities["MLB"])):
            if mlb_df.iloc[i]["label"] in cities["MLB"][j]:
                mlb_areas.append(cities["Metropolitan area"][j])
    mlb_df["Metropolitan area"] = mlb_areas

    # Win/Loss ratio
    mlb_df["Win_Loss_Ratio"] = mlb_df["W"] / (mlb_df["W"] + mlb_df["L"])

    mlb_df_question = mlb_df.groupby("Metropolitan area")["Win_Loss_Ratio"].mean()
    result = pd.merge(mlb_df_question.reset_index(), cities[["Metropolitan area", "Population (2016 est.)"]],
                       how="left", on="Metropolitan area")


    population_by_region = list(result["Population (2016 est.)"]) # pass in metropolitan area population from cities
    win_loss_by_region = list(result["Win_Loss_Ratio"]) # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.15027698302669307

## The win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [33]:
import pandas as pd
import scipy.stats as stats

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns={"Population (2016 est.)[8]" : "Population (2016 est.)"}, inplace=True)
    cities["Population (2016 est.)"] = cities["Population (2016 est.)"].map(lambda x: float(x))
    cities.replace(r"\[note [\d][\d]?\]", "", regex=True, inplace=True)

    nfl_df = nfl_df[nfl_df["year"] == 2018]
    nfl_df.drop(index=[0, 5, 10, 15, 20, 25, 30, 35], inplace=True)

    # convert str to numbers for dedicated columns
    colums_to_float = ['DSRS', 'L', 'MoV', 'OSRS', 'PA', 'PD', 'PF', 'SRS', 'SoS', 'T', 'W', 'W-L%']
    nfl_df[colums_to_float] = nfl_df[colums_to_float].apply(lambda x: x.map(lambda y : float(y)))
    nfl_df["team"].replace("\*|\+", "", regex=True, inplace=True)

    pattern_2 = r"(?P<Metropolitan_area>.+ )(?P<team_label>[A-Z\d][a-z\d]+)"
    nfl_df["label"] = nfl_df["team"].str.extract(pattern_2)["team_label"]
    nfl_df["Win_Loss_Ratio"] = nfl_df["W"] / (nfl_df["W"] + nfl_df["L"])
    nfl_areas = []
    for i in range(len(nfl_df["label"])):
        for j in range(len(cities["NFL"])):
            if nfl_df.iloc[i]["label"] in cities["NFL"][j]:
                nfl_areas.append(cities["Metropolitan area"][j])
    nfl_df["Metropolitan area"] = nfl_areas

    nfl_df_question = nfl_df.groupby("Metropolitan area")["Win_Loss_Ratio"].mean()
    result = pd.merge(nfl_df_question.reset_index(), cities[["Metropolitan area", "Population (2016 est.)"]],
                   how="left", on="Metropolitan area")
    
    population_by_region = list(result["Population (2016 est.)"]) # pass in metropolitan area population from cities
    win_loss_by_region = list(result["Win_Loss_Ratio"]) # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004922112149349409

## Here we explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests between all pairs of sports. Are there any sports where we can reject the null hypothesis? Average values where a sport has multiple teams in one region.

In [34]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    # Preparing wiki data
    cities = pd.read_html("./assets/wikipedia_data.html")
    cities = cities[1].iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns={"Population (2016 est.)[8]" : "Population (2016 est.)"}, inplace=True)
    cities["Population (2016 est.)"] = cities["Population (2016 est.)"].map(lambda x: float(x))
    cities.replace(r"\[note [\d][\d]?\]", "", regex=True, inplace=True)

    # NFL preparation
    nfl_df = pd.read_csv("./assets/nfl.csv")
    nfl_df = nfl_df[nfl_df["year"] == 2018]
    nfl_df.drop(index=[0, 5, 10, 15, 20, 25, 30, 35], inplace=True)

    # convert str to numbers for dedicated columns
    colums_to_float = ['DSRS', 'L', 'MoV', 'OSRS', 'PA', 'PD', 'PF', 'SRS', 'SoS', 'T', 'W', 'W-L%']
    nfl_df[colums_to_float] = nfl_df[colums_to_float].apply(lambda x: x.map(lambda y : float(y)))
    nfl_df["team"].replace("\*|\+", "", regex=True, inplace=True)

    pattern_2 = r"(?P<Metropolitan_area>.+ )(?P<team_label>[A-Z\d][a-z\d]+)"
    nfl_df["label"] = nfl_df["team"].str.extract(pattern_2)["team_label"]
    cities.replace(r"\[note [\d][\d]?\]", "", regex=True, inplace=True)

    nfl_areas = []
    for i in range(len(nfl_df["label"])):
        for j in range(len(cities["NFL"])):
            if nfl_df.iloc[i]["label"] in cities["NFL"][j]:
                nfl_areas.append(cities["Metropolitan area"][j])
    nfl_df["Metropolitan area"] = nfl_areas

    nfl_df["Win_Loss_Ratio"] = nfl_df["W"] / (nfl_df["W"] + nfl_df["L"])

    # NBA preparation
    nba_df = pd.read_csv("./assets/nba.csv")
    nba_df = nba_df[nba_df["year"] == 2018]
    nba_df["team"].replace("\*?\s\([\d][\d]?\)", "", regex=True, inplace=True)

    pattern = r"(?P<Metropolitan_area>.+ )(?P<team_label>[A-Z\d][a-z\d]+)"
    nba_df["label"] = nba_df["team"].str.extract(pattern)["team_label"]

    # Setting Metropolitan area
    nba_areas = []
    for i in range(len(nba_df["label"])):
        for j in range(len(cities["NBA"])):
            if nba_df.iloc[i]["label"] in cities["NBA"][j]:
                nba_areas.append(cities["Metropolitan area"][j])
    nba_df["Metropolitan area"] = nba_areas

    # Win/Loss ratio
    # convert str to numbers for dedicated columns
    colums_to_float = ["L", "W"]
    nba_df[colums_to_float] = nba_df[colums_to_float].apply(lambda x: x.map(lambda y : float(y)))
    nba_df["Win_Loss_Ratio"] = nba_df["W"] / (nba_df["W"] + nba_df["L"])
    
    # NHL preparation
    nhl_df = pd.read_csv("./assets/nhl.csv")
    nhl_df = nhl_df[nhl_df["year"] == 2018]
    nhl_df.drop(index=[0, 9, 18, 26], inplace=True)
    nhl_df["team"].replace("\*", "", regex=True, inplace=True)

    pattern = r"(?P<Metropolitan_area>.+ )(?P<team_label>[A-Z\d][a-z\d]+)"
    nhl_df["label"] = nhl_df["team"].str.extract(pattern)["team_label"]

    # Setting Metropolitan area
    nhl_areas = []
    for i in range(len(nhl_df["label"])):
        for j in range(len(cities["NHL"])):
            if nhl_df.iloc[i]["label"] in cities["NHL"][j]:
                nhl_areas.append(cities["Metropolitan area"][j])
    nhl_df["Metropolitan area"] = nhl_areas

    # Win/Loss ratio
    # convert str to numbers for dedicated columns
    colums_to_float = ["L", "W"]
    nhl_df[colums_to_float] = nhl_df[colums_to_float].apply(lambda x: x.map(lambda y : float(y)))
    nhl_df["Win_Loss_Ratio"] = nhl_df["W"] / (nhl_df["W"] + nhl_df["L"])

    # MBL preparation
    mlb_df = pd.read_csv("./assets/mlb.csv")
    mlb_df = mlb_df[mlb_df["year"] == 2018]

    pattern = r"(?P<Metropolitan_area>.+ )(?P<team_label>[A-Z\d][a-z\d]+)"
    mlb_df["label"] = mlb_df["team"].str.extract(pattern)["team_label"]
    # Label correction:
    mlb_df.at[0, "label"] = "Red Sox"
    mlb_df.at[8, "label"] = "White Sox"

    # Setting Metropolitan area
    mlb_areas = []
    for i in range(len(mlb_df["label"])):
        for j in range(len(cities["MLB"])):
            if mlb_df.iloc[i]["label"] in cities["MLB"][j]:
                mlb_areas.append(cities["Metropolitan area"][j])
    mlb_df["Metropolitan area"] = mlb_areas

    # Win/Loss ratio
    mlb_df["Win_Loss_Ratio"] = mlb_df["W"] / (mlb_df["W"] + mlb_df["L"])

    nfl_df_question = nfl_df.groupby("Metropolitan area")["Win_Loss_Ratio"].mean()
    nba_df_question = nba_df.groupby("Metropolitan area")["Win_Loss_Ratio"].mean()
    nhl_df_question = nhl_df.groupby("Metropolitan area")["Win_Loss_Ratio"].mean()
    mlb_df_question = mlb_df.groupby("Metropolitan area")["Win_Loss_Ratio"].mean()

    result_nfl_nba = pd.merge(nfl_df_question.reset_index(), nba_df_question.reset_index(),
                              how="inner", on="Metropolitan area")
    result_nfl_nhl = pd.merge(nfl_df_question.reset_index(), nhl_df_question.reset_index(),
                              how="inner", on="Metropolitan area")
    result_nfl_mlb = pd.merge(nfl_df_question.reset_index(), mlb_df_question.reset_index(),
                              how="inner", on="Metropolitan area")
    result_nba_nhl = pd.merge(nba_df_question.reset_index(), nhl_df_question.reset_index(),
                              how="inner", on="Metropolitan area")
    result_nba_mlb = pd.merge(nba_df_question.reset_index(), mlb_df_question.reset_index(),
                              how="inner", on="Metropolitan area")
    result_nhl_mlb = pd.merge(nhl_df_question.reset_index(), mlb_df_question.reset_index(),
                              how="inner", on="Metropolitan area")
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    p_values.at["NFL", "NBA"] = stats.ttest_rel(result_nfl_nba["Win_Loss_Ratio_x"],
                                                 result_nfl_nba["Win_Loss_Ratio_y"])[1]
    p_values.at["NBA", "NFL"] = p_values.at["NFL", "NBA"]
    p_values.at["NFL", "NHL"] = stats.ttest_rel(result_nfl_nhl["Win_Loss_Ratio_x"],
                                                 result_nfl_nhl["Win_Loss_Ratio_y"])[1]
    p_values.at["NHL", "NFL"] = p_values.at["NFL", "NHL"]
    p_values.at["NFL", "MLB"] = stats.ttest_rel(result_nfl_mlb["Win_Loss_Ratio_x"],
                                                 result_nfl_mlb["Win_Loss_Ratio_y"])[1]
    p_values.at["MLB", "NFL"] = p_values.at["NFL", "MLB"]
    p_values.at["NBA", "NHL"] = stats.ttest_rel(result_nba_nhl["Win_Loss_Ratio_x"],
                                                 result_nba_nhl["Win_Loss_Ratio_y"])[1]
    p_values.at["NHL", "NBA"] = p_values.at["NBA", "NHL"]
    p_values.at["NBA", "MLB"] = stats.ttest_rel(result_nba_mlb["Win_Loss_Ratio_x"],
                                                 result_nba_mlb["Win_Loss_Ratio_y"])[1]
    p_values.at["MLB", "NBA"] = p_values.at["NBA", "MLB"]
    p_values.at["NHL", "MLB"] = stats.ttest_rel(result_nhl_mlb["Win_Loss_Ratio_x"],
                                                 result_nhl_mlb["Win_Loss_Ratio_y"])[1]
    p_values.at["MLB", "NHL"] = p_values.at["NHL", "MLB"]

    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
